### QAPQ FORWARD PASS

0. load model

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/root/data/models/lice-classification/QAPQ/v1/model.49.hdf5')

1. select one crop with enough annotations using the db

In [ ]:
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import cv2

In [ ]:
fish_id = 1211501
key = 'dev/blom-kjeppevikholmen/2/2019-04-02/left_blom-kjeppevikholmen_2_1554211262082_690_1018_4018_2503.jpg'
local_path = '/root/data/lice-data/crops/blom-kjeppevikholmen/2/2019-04-02/left_blom-kjeppevikholmen_2_1554211262082_690_1018_4018_2503.jpg'

In [ ]:
cogito_ann = [{"id":2,"category":"MOVING","position":{"left":756,"top":371,"width":13,"height":15},"location":"MIDDLE"},
               {"id":3,"category":"MOVING","position":{"left":768,"top":380,"width":32,"height":22},"location":"MIDDLE"},
               {"id":4,"category":"MOVING","position":{"left":788,"top":342,"width":16,"height":11},"location":"MIDDLE"},
               {"id":5,"category":"MOVING","position":{"left":807,"top":346,"width":18,"height":13},"location":"MIDDLE"},
               {"id":6,"category":"MOVING","position":{"left":767,"top":160,"width":15,"height":13},"location":"MIDDLE"},
               {"id":7,"category":"ADULT_FEMALE","position":{"left":741,"top":527,"width":46,"height":29},"location":"MIDDLE"},
               {"id":8,"category":"MOVING","position":{"left":772,"top":480,"width":22,"height":14},"location":"MIDDLE"},
               {"id":9,"category":"MOVING","position":{"left":407,"top":184,"width":16,"height":11},"location":"MIDDLE"},
               {"id":10,"category":"MOVING","position":{"left":2404,"top":602,"width":17,"height":9},"location":"TOP"},
               {"id":11,"category":"MOVING","position":{"left":2528,"top":669,"width":19,"height":12},"location":"TOP"},
               {"id":12,"category":"MOVING","position":{"left":2677,"top":751,"width":17,"height":10},"location":"TOP"},
               {"id":13,"category":"ADULT_FEMALE","position":{"left":2843,"top":861,"width":49,"height":26},"location":"TOP"},
               {"id":14,"category":"MOVING","position":{"left":2903,"top":902,"width":12,"height":8},"location":"TOP"},
               {"id":15,"category":"ADULT_FEMALE","position":{"left":2812,"top":1155,"width":44,"height":19},"location":"BOTTOM"}
              ]
print('Number of lice detected by cogito: {}'.format(len(cogito_ann)))

In [ ]:
gunnar_ann = json.load(open(local_path.replace('jpg', 'json')))['detections']
print('Number of lice detected by gunnar: {}'.format(len(gunnar_ann)))

2. some display

In [ ]:
image = cv2.imread(local_path)

In [ ]:
# 184 407
# 480 772

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(image)
for cann in cogito_ann:
    position = cann['position']
    x1 = position['top']
    y1 = position['left']
    # print(x1, y1)
    width = position['width']
    height=position['height']
    rec = Rectangle((y1, x1), width, height, facecolor='none', edgecolor='b', linewidth=2)
    ax.add_patch(rec)
# plt.axis('off')
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(image)

for gann in gunnar_ann:
    position = gann['position']
    x1 = position['top']
    y1 = position['left']
    width = position['width']
    height=position['height']
    rec = Rectangle((y1, x1), width, height, facecolor='none', edgecolor='r', linewidth=2)
    ax.add_patch(rec)
plt.axis('off')
plt.show()

3. select cogito crops and forward pass the model

In [ ]:
from albumentations import PadIfNeeded, HorizontalFlip, VerticalFlip, Compose, RandomRotate90

In [ ]:
mapping = {'ADULT_FEMALE':0,
           'FIXED': 1,
           'MOVING': 1}

In [ ]:
input_shape = [64, 64, 3]

In [ ]:
def load_lice(image):
    '''load lice'''
    aug = Compose([PadIfNeeded(p=1.0, min_height=input_shape[0], min_width=input_shape[1], border_mode=0)])
    height, width, _ = image.shape
    if height > width:
        ratio = input_shape[0] / float(height)
        x = cv2.resize(image, (int(width*ratio), input_shape[1]), interpolation=cv2.INTER_LINEAR)
    else:
        ratio = input_shape[1] / float(width)
        x = cv2.resize(image, (input_shape[0], int(height*ratio)), interpolation=cv2.INTER_LINEAR)
    augmented = aug(image=x)
    x = augmented["image"]
    return x 

In [ ]:
fp = 0
for cann in cogito_ann:
    cat = cann['category']
    position = cann['position']
    x1 = position['top']
    y1 = position['left']
    width = position['width']
    height=position['height']
    lice = image[x1:x1+height, y1:y1+width, :]
    model_input = load_lice(lice)

    preds = model.predict_on_batch(np.expand_dims(model_input, 0)).squeeze()
    print(preds)
    if np.argmax(preds) != mapping[cat]:
        fp += 1
score = fp / len(cogito_ann)
print('score: {}'.format(score))